In [2]:
#imports
import pandas as pd
from google.cloud import bigquery
from time import time
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering/week_3_data_warehouse/GOOGLE_APPLICATION_CREDENTIALS.json"
print("Setup Complete")

Setup Complete


In [4]:
#create a client
client = bigquery.Client()
# Construct a reference to the "ny_taxi_bucket_de_2023" dataset
dataset_ref = client.dataset(dataset_id="ny_taxi", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#construct a table reference to "green_tripdata_2019-11"
table_ref = dataset_ref.table('green_tripdata_2019-11')
#API request
table_green_taxi_trips = client.get_table(table_ref)
#to dataframe
df_yellow_taxi_trips = client.list_rows(table=table_green_taxi_trips, max_results=5).to_dataframe()
df_yellow_taxi_trips.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-11-01 00:04:42+00:00,2019-11-01 00:13:53+00:00,N,1.0,95,196,1.0,1.51,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,1.0,0.0
1,2,2019-11-01 00:43:35+00:00,2019-11-01 00:43:39+00:00,N,1.0,193,7,1.0,0.00,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,1.0,0.0
2,1,2019-11-01 03:10:58+00:00,2019-11-01 03:13:59+00:00,N,5.0,129,129,1.0,0.70,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,2.0,0.0
3,1,2019-11-01 03:46:32+00:00,2019-11-01 03:54:20+00:00,N,5.0,129,173,1.0,1.00,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,2.0,0.0
4,1,2019-11-01 04:03:12+00:00,2019-11-01 04:07:50+00:00,N,5.0,129,129,1.0,0.60,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,2.0,0.0


In [59]:
# query
query = """
        SELECT COUNT(1) as num_count
        FROM `dtc-de-2023.prefect_de_2023_datasetid.yellow_ny_taxi_trips_table`
        WHERE DATE(tpep_pickup_datetime) BETWEEN "2021-01-01" AND "2021-01-01"
        LIMIT 100
        """
        
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
query_results = client.query(query, job_config=safe_config)
#convert to dataframe
df_query_results = query_results.to_dataframe()
df_query_results.head()

client.create_table()

,num_count
0,24827


(1, 1)

In [22]:
# amount of data scanned in GB
def data_scanned(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job_bytes = client.query(query, job_config=dry_run_config).total_bytes_processed
    print(f'Data processed: {round(query_job_bytes / 10**9, 3)} GB')

def runtime(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print(f'Runtime is: {round(end-start, 3)} seconds')

In [23]:
data_scanned(query=query)
runtime(query=query)

Data processed: 0.011 GB
Runtime is: 1.568 seconds


In [ ]:
# Construct a reference to the "prefect_de_2023" dataset
dataset_ref_gcs = bigquery.DatasetReference(project="dtc-de-2023", dataset_id="prefect-de-2023")
# construct a table reference to "green_tripdata_2019-04"
table_ref_gcs = dataset_ref_gcs.table("green_tripdata_2020-01")
# API request
green_ny_taxi_trips_table = bigquery.Table(table_ref_gcs)
external_config = bigquery.ExternalConfig("PARQUET")
external_config.source_uris = ["gs://prefect-de-2023/data/green/green_tripdata_2020-01.parquet"]
# green_ny_taxi_trips_table.external_data_configuration = external_config

# Create a permanent table linked to GCS file
table = client.create_table(table=green_ny_taxi_trips_table, )
# to DataFrame
df_green_ny_taxi_trips = client.list_rows(table=table, max_results=5).to_dataframe()
df_green_ny_taxi_trips.head()

# 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/datasets/prefect-de-2023/tables?prettyPrint=false: Not found: Dataset dtc-de-2023:prefect-de-2023

In [ ]:
#create a client
client = bigquery.Client()
# Construct a reference to the "prefect_de_2023_datasetid" dataset
dataset_ref = client.dataset("prefect_de_2023_datasetid", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
# construct a table reference to "yellow_ny_taxi_trips_table"
table_ref = dataset_ref.table('yellow_ny_taxi_trips_table')
#API request
table_yellow_taxi_trips = client.get_table(table_ref)
#to dataframe
df_yellow_taxi_trips = client.list_rows(table=table_yellow_taxi_trips, max_results=5).to_dataframe()
df_yellow_taxi_trips.head()

In [57]:
# Downloads a blob to a file path.
from prefect import flow
from prefect_gcp import GcpCredentials
from prefect_gcp.cloud_storage import cloud_storage_download_blob_to_file
from pathlib import Path
@flow
def example_cloud_storage_download_blob_flow():
    gcp_credentials = GcpCredentials.load("ny-taxi-gcp-creds")  # inferred from env, or set service_account_file
    contents = cloud_storage_download_blob_to_file(bucket="ny_taxi_bucket_de_2023", blob="green/green_tripdata_2019-11.parquet", gcp_credentials=gcp_credentials, path=Path("./week3_hw"))
    return contents

example_cloud_storage_download_blob_flow()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'example-cloud-storage-download-blob-flow' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_71701/1514260246.py:6' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


11:58:25.619 | INFO    | prefect.engine - Created flow run 'authentic-otter' for flow 'example-cloud-storage-download-blob-flow'

11:58:28.950 | INFO    | Flow run 'authentic-otter' - Created task run 'cloud_storage_download_blob_to_file-7cd7d622-0' for task 'cloud_storage_download_blob_to_file'

11:58:28.952 | INFO    | Flow run 'authentic-otter' - Executing 'cloud_storage_download_blob_to_file-7cd7d622-0' immediately...

11:58:30.221 | INFO    | Task run 'cloud_storage_download_blob_to_file-7cd7d622-0' - Downloading blob named green/green_tripdata_2019-11.parquet from the ny_taxi_bucket_de_2023 bucket to week3_hw

11:58:38.717 | INFO    | Task run 'cloud_storage_download_blob_to_file-7cd7d622-0' - Finished in state Completed()

11:58:39.077 | INFO    | Flow run 'authentic-otter' - Finished in state Completed()

PosixPath('week3_hw/green/green_tripdata_2019-11.parquet')

In [122]:
# read data from the web into Dataframe
from pathlib import Path
import urllib.request

dataset_url = (
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-04.parquet"
    )

def fetch(dataset_url):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


In [123]:
fetch(dataset_url)

'/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/tmpk_dytf59'

In [125]:
def read_df(path: str) -> pd.DataFrame:
    if dataset_url.endswith(".parquet"):
        df = pd.read_parquet(path=path)
    elif dataset_url.endswith(".csv.gz"):
        df = pd.read_csv(path, engine="pyarrow", compression="gzip")
    else:
        df = pd.read_csv(path, engine="pyarrow")
    
    print(f"Number of rows: {df.shape[0]}")
    return df

path_name = fetch(dataset_url=dataset_url)
df_ = read_df(path_name)
df_

Number of rows: 238073


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2020-04-01 00:41:22,2020-04-01 01:01:53,1.0,1.20,1.0,N,41,24,2,5.50,0.5,0.5,0.00,0.00,0.3,6.80,0.0,None
1,1,2020-04-01 00:56:00,2020-04-01 01:09:25,1.0,3.40,1.0,N,95,197,1,12.50,0.5,0.5,2.75,0.00,0.3,16.55,0.0,None
2,1,2020-04-01 00:00:26,2020-04-01 00:09:25,1.0,2.80,1.0,N,237,137,1,10.00,3.0,0.5,1.00,0.00,0.3,14.80,2.5,None
3,1,2020-04-01 00:24:38,2020-04-01 00:34:38,0.0,2.60,1.0,N,68,142,1,10.00,3.0,0.5,1.00,0.00,0.3,14.80,2.5,None
4,2,2020-04-01 00:13:24,2020-04-01 00:18:26,1.0,1.44,1.0,Y,263,74,1,6.50,0.5,0.5,3.00,0.00,0.3,13.30,2.5,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238068,2,2020-04-30 23:30:00,2020-04-30 23:39:00,NaN,2.02,NaN,None,238,75,0,8.46,0.0,0.5,0.00,0.00,0.3,11.76,NaN,None
238069,2,2020-04-30 23:44:00,2020-04-30 23:54:00,NaN,3.30,NaN,None,263,230,0,13.46,0.0,0.5,0.00,0.00,0.3,16.76,NaN,None
238070,2,2020-04-30 23:25:00,2020-04-30 23:38:00,NaN,6.40,NaN,None,137,42,0,22.05,0.0,0.5,0.00,0.00,0.3,25.35,NaN,None
238071,2,2020-04-30 23:50:26,2020-05-01 00:28:21,NaN,25.70,NaN,None,29,69,0,68.16,0.0,0.5,0.00,6.12,0.3,77.58,NaN,None


In [91]:
type(path)

str

In [104]:
# 
from prefect import flow,task
from prefect_gcp import GcpCredentials
from prefect_gcp.cloud_storage import cloud_storage_upload_blob_from_file, c
from pathlib import Path

@flow
def example_cloud_storage_upload_blob_flow():
    path = fetch(dataset_url)
    gcp_credentials = GcpCredentials.load("ny-taxi-gcp-creds")  # inferred from env, or set service_account_file
    blob = cloud_storage_upload_blob_from_file(file=path, 
                                                bucket="ny_taxi_bucket_de_2023",
                                                blob="green/green_tripdata_2020-04.parquet",
                                                gcp_credentials=gcp_credentials)
    return blob

example_cloud_storage_upload_blob_flow()


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'example-cloud-storage-upload-blob-flow' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_71701/4259268297.py:7' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


13:02:29.544 | INFO    | prefect.engine - Created flow run 'precious-grouse' for flow 'example-cloud-storage-upload-blob-flow'

13:02:42.951 | INFO    | Flow run 'precious-grouse' - Created task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' for task 'cloud_storage_upload_blob_from_file'

13:02:42.959 | INFO    | Flow run 'precious-grouse' - Executing 'cloud_storage_upload_blob_from_file-9a1bf371-0' immediately...

13:02:44.136 | INFO    | Task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' - Uploading blob named green/green_tripdata_2020-04.parquet to the ny_taxi_bucket_de_2023 bucket

13:02:48.315 | INFO    | Task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' - Finished in state Completed()

13:02:48.665 | INFO    | Flow run 'precious-grouse' - Finished in state Completed()

'green/green_tripdata_2020-04.parquet'